In [31]:
import pandas as pd
df=pd.read_csv('/Users/abhaysingh/Documents/Resume/data/Cleaned_resume.csv')

In [32]:
df.head()
df.info()
df["Category"].unique()
df["Category"].info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 962 entries, 0 to 961
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Category        962 non-null    object
 1   cleaned_resume  962 non-null    object
dtypes: object(2)
memory usage: 15.2+ KB
<class 'pandas.core.series.Series'>
RangeIndex: 962 entries, 0 to 961
Series name: Category
Non-Null Count  Dtype 
--------------  ----- 
962 non-null    object
dtypes: object(1)
memory usage: 7.6+ KB


In [33]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['Category']=le.fit_transform(df['Category'])
df.head()
X=df['cleaned_resume']
y=df['Category']

In [34]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(max_features=5000)
X_train_tf=tfidf.fit_transform(X_train)
X_test_tf=tfidf.transform(X_test)
X_train_tf.shape

(769, 5000)

In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score

In [37]:
class MyDataset(Dataset):
  def __init__(self,features,labels):
    self.features=torch.tensor(features,dtype=torch.float32)
    self.labels=torch.tensor(labels,dtype=torch.long)
  def __len__(self):
    return len(self.features)
  def __getitem__(self,idx):
    return self.features[idx],self.labels[idx]

In [38]:
# as TF-IDF gives a sparce matrix but pytorch doesnt take in the form of matrix
X_train=X_train_tf.toarray()
X_test=X_test_tf.toarray()
X_train.shape         # (769,5000). 769 ---> number of samples ,,,,,, 5000--> number of features

(769, 5000)

In [39]:
train_dataset=MyDataset(X_train,y_train.values) # as y_train is a pandas series we have to make it into numpys arrays
test_dataset=MyDataset(X_test,y_test.values)

# now we are going to make our loaders so that we can give training in batches we use 32 batch
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False)

In [40]:
num_classes = df['Category'].nunique()
print(num_classes)

25


In [41]:
import torch.nn.functional as F
class MyNN(nn.Module):
  def __init__(self,num_features):
    super().__init__()

    self.model=nn.Sequential(
        nn.Linear(num_features,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,25)
    )
  def forward(self,X):
    return self.model(X)

In [42]:
# Input TF-IDF vector (5000-d) 
#         ↓ Linear + ReLU
# Hidden Layer 128-d
#         ↓ Linear + ReLU
# Hidden Layer 64-d
#         ↓ Linear
# Output Layer 25-d logits → predicted scores for each class

In [43]:
# lets decide the learning rate and epoch size
l_rate=0.01
epochs=100
criterion=nn.CrossEntropyLoss() # loss function of the model
model=MyNN(X_train.shape[1])  # instance of the model
optimizer=optim.Adam(model.parameters(),lr=l_rate)

In [44]:
for epoch in range(epochs):
    total_epoch_loss=0
    for batch_features,batch_labels in train_loader:

        # forward pass
        num_predict=model(batch_features)

        # loss
        loss=criterion(num_predict,batch_labels)

        #backpropagate with zero grad
        optimizer.zero_grad()
        loss.backward()

        #optimize.step() update weights
        optimizer.step()

        # calculate the loss 
        total_epoch_loss=total_epoch_loss+loss.item()
  
    avg_loss=total_epoch_loss/len(train_loader)
    print(f"Epoch{epoch+1},Loss:{avg_loss}")


Epoch1,Loss:2.125615677833557
Epoch2,Loss:0.12495484890416236
Epoch3,Loss:0.0010120257321975146
Epoch4,Loss:0.000263427312602289
Epoch5,Loss:0.00010071021313706296
Epoch6,Loss:7.746503109956393e-05
Epoch7,Loss:6.388268357113703e-05
Epoch8,Loss:7.629207895661239e-05
Epoch9,Loss:5.3890456183580684e-05
Epoch10,Loss:4.4173746791784653e-05
Epoch11,Loss:3.745379166502971e-05
Epoch12,Loss:3.400053306904738e-05
Epoch13,Loss:3.1362008576252266e-05
Epoch14,Loss:2.8955932448297973e-05
Epoch15,Loss:2.9937638646515553e-05
Epoch16,Loss:2.477666179402149e-05
Epoch17,Loss:2.7503595265443437e-05
Epoch18,Loss:2.1599045858238242e-05
Epoch19,Loss:2.093196973874001e-05
Epoch20,Loss:1.9275581908004825e-05
Epoch21,Loss:1.7794986997614616e-05
Epoch22,Loss:1.735806163196685e-05
Epoch23,Loss:1.5897852753141704e-05
Epoch24,Loss:1.505602513134363e-05
Epoch25,Loss:1.4498048894893145e-05
Epoch26,Loss:1.3630715980070818e-05
Epoch27,Loss:1.3071236171526835e-05
Epoch28,Loss:1.2476876772780088e-05
Epoch29,Loss:1.279102

In [45]:
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=5000, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=25, bias=True)
  )
)

In [46]:
# Initialize counters
total = 0        # Total number of test samples
correct = 0      # Total number of correct predictions

# Disable gradient calculation during evaluation
# This makes evaluation faster and uses less memory
with torch.no_grad():  
    # Loop over the test dataset in batches
    for batch_features, batch_labels in test_loader:
        
        # Forward pass: get model outputs (logits for each class)
        pred_output = model(batch_features)
        
        # Get predicted class for each sample
        # torch.max returns (max_value, index_of_max)
        # We only need the index which corresponds to predicted class
        _, predicted = torch.max(pred_output, dim=1)
        
        # Update total number of samples seen
        total += batch_labels.size(0)
        
        # Count correct predictions in this batch and update total correct
        correct += (predicted == batch_labels).sum().item()

# Calculate overall accuracy as percentage
accuracy_score = (correct / total) * 100

# Print test accuracy
print(f"Accuracy: {accuracy_score}%")


Accuracy: 99.48186528497409%


In [47]:
# as we achieve a good accuracy now it the time to save the model
torch.save(model.state_dict(),"model.pth")


In [48]:
# we are now going to save the vectorized resumes in the pickle file 
import pickle
with open("tfidf_vectorizer.pkl","wb") as f:
    pickle.dump(tfidf,f)

In [49]:
with open("Label_encoding.pkl","wb") as f:
    pickle.dump(le,f)